In [1]:
# %%
import os
import glob
import librosa
# import librosa.display as ldp
# import IPython.display as idp
import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns
# import tensorflow as tf
# from sklearn.svm import SVC
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from collections import defaultdict
import re
from math import log2

In [2]:

# import cv2
# import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization
# from sklearn.model_selection import train_test_split
# from sklearn import metrics


# %%
morse_table = {}
morse_table = defaultdict(lambda: '?',morse_table)
morse_table = {
      '.-':'A', '-...':'B', '-.-.':'C', '-..':'D', '.':'E',
      '..-.':'F', '--.':'G', '....':'H', '..':'I', '.---':'J',
      '-.-':'K', '.-..' : 'L', '--' :'M', '-.' :'N', '---':'O',
      '.--.' : 'P', '--.-' : 'Q', '.-.':'R', '...':'S', '-'  :'T',
      '..-':'U', '...-' : 'V', '.--':'W', '-..-' : 'X', '-.--' : 'Y',
      '--..' : 'Z', '.----' : '1', '..---' : '2', '...--' : '3', 
      '....-' : '4', '.....' : '5', '-....' : '6', '--...' : '7', 
      '---..' : '8','----.' : '9','-----' : '0',
      '-...-' : '=', '.-.-':'~', '.-...' :'<AS>', '.-.-.' : '<AR>', '...-.-' : '<SK>',
      '-.--.' : '<KN>', '..-.-' : '<INT>', '....--' : '<HM>', '...-.' : '<VE>',
      '.-..-.' : '\\', '.----.' : '\'', '...-..-' : '$', '-.--.' : '(', '-.--.-' : ')', 
      '--..--' : ',', '-....-' : '-', '.-.-.-' : '.', '-..-.' : '/', '---...' : ':', 
      '-.-.-.' : ';', '..--..' : '?', '..--.-' : '_', '.--.-.' : '@', '-.-.--' : '!'
    }
morse_table.setdefault('?')
'''
def morse_to_text(morse_code):
    res = ""
    tmp = ''
    morse_code += ' '
    for i in range(len(morse_code)):
        c = morse_code[i]
        if c == ' ':
            if (len(tmp)>0):
                if (len(tmp) > 5):
                    tmp = tmp[:5]
                while (len(tmp) < 5):
                    if (tmp[0] == '-'):
                        tmp = '-' + tmp
                    else:
                        tmp = tmp + '-'
                if (tmp in morse_table):
                    res += morse_table[tmp]
                else:
                    res += '?'
            tmp = ''
            res += ' '
        else:
            tmp += c
    return res'''
def morse_to_text(morse_code):
    res = ""
    tmp = ''
    morse_code += ' '
    for i in range(len(morse_code)):
        c = morse_code[i]
        if c == ' ':
            if (len(tmp)>0):
                if (tmp in morse_table):
                    res += morse_table[tmp]
                else:
                    res += '?'
            tmp = ''
            res += ' '
        else:
            tmp += c
    return res

# %%
def get_index(start,end,step):
    return list(range(round(start/step),round(end/step)))
    
def gan_nhan (link_file,step):
    res = []
    with open(link_file) as f:
        for line in f:
            start,end,label = line.split('\t')
            start = float(start)
            end = float(end)
            step = float(step)
            res.extend(get_index(start,end,step))
    return(res)

# %%
#rms, spectral_bandwidth(birch), spectral_rolloff(agg,miniKmeans)
#MiniBatchKMeans, KMeans: tonnetz 92%, spectral_rolloff 70%, spectral_bandwidth 72%, melspectrogram 85%, 
#spectral_rolloff 
def read(wav_file):
    step = 0.0
    signal,sr = librosa.load(wav_file)
    tonnetz = librosa.feature.tonnetz(y=signal,sr=sr, hop_length=128)
    #delta_tonnetz = librosa.feature.delta(tonnetz)
    #melspectrogram = librosa.feature.melspectrogram(signal,sr=sr, hop_length=128)
    features = tonnetz#np.concatenate((tonnetz))
    duration = librosa.get_duration(y=signal,sr=sr)
    num_tonnetz_cens = features.shape[1]
    step = duration/num_tonnetz_cens
    x = features.transpose()
    return x,step
    

# %%
def rename_folder(link_folder):
    for filename in glob.glob(link_folder+'*.wav'):
        os.rename(filename,filename.replace(' ','_'))


# %%
def predict_morse(pred,step):
    len_space = int(log2(step))
    counter = [0,0]
    equal_seq = 1
    res = ""
    counter[pred[0]] = 1
    for i in range(1,len(pred)):
        if (pred[i] == pred[i-1]):
            equal_seq += 1
            counter[pred[i]] += 1
        else:
            #if equal_seq >2:
            if counter[1] > 13:
                res += '-'
            elif counter[1] > 0:
                res += '.'
            elif counter[0] > 22:
                res += ' '* (counter[0] // 21)
            
            counter = [0,0]
            counter[pred[i]] = 1
            
            equal_seq = 1
    #res = re.sub(' +',' ',res)
    return res

# %%
def flatten_pred(pred):
    fat_pred = pred
    counter = 0
    for i in range(1,len(pred)-1):
        if fat_pred[i-1] == 1 and fat_pred[i]==0 and fat_pred[i+1]==1:
            fat_pred[i] = 0
        elif i+2<len(pred):
            if fat_pred[i-1] == 1 and fat_pred[i]==0 and fat_pred[i+1]==0 and fat_pred[i+2] ==0:
                fat_pred[i]=0
                fat_pred[i+1]=0
    return fat_pred

# %%
#every 20s

def MiniKMeans(Xt):
    while True:
        minikmeans =MiniBatchKMeans(n_clusters=2,n_init=2,reassignment_ratio=0.5).fit(Xt)
        pred = minikmeans.labels_
        if pred[0]==0:
            break
    return pred

# %%
def pred_to_label(pred,step,filename):
    fo = open(filename,'w')
    start = 0
    for i in range(1,len(pred)):
        if (pred[i] == 1 and pred[i-1]==0):
            start = i
        elif pred[i]==0 and pred[i-1] ==1 :
            start_time = start*step
            end_time = i*step
            fo.write('{}\t{}\t1\n'.format(start_time,end_time))
    fo.close()


# %%
def predict_folder(link_folder,output_folder):
    for filename in glob.glob(link_folder+'*.wav'):
        os.rename(filename,filename.replace(' ','_'))
    for wav in glob.glob(link_folder+'*.wav'):
        X, step = read(wav)
        
        while True:
            minikmeans =MiniBatchKMeans(n_clusters=2,n_init=2,reassignment_ratio=0.5).fit(X)
            pred = minikmeans.labels_
            if pred[0]==0:
                break
        text = predict_morse(pred,step)
        output_filename = os.path.basename(wav)[:-4]+'.txt'
        
        fo = open(output_folder+'text/'+output_filename,'w')
        fo.write(morse_to_text(text))
        fo.close()
        
        fo = open(output_folder+'code/'+output_filename,'w')
        fo.write(text)
        fo.close()
        
        pred_to_label(pred,step,link_folder+output_filename)

# %%

In [7]:

X,step = read(r'/home/morse_sound.wav')

# %%
pred = MiniKMeans(X)

flaten_pred = predict_morse(flatten_pred(pred),step)
straight_pred = predict_morse(pred,step)
print('flaten_pred ',flaten_pred)
print('straight_pred ',straight_pred)

# %%
while True:
    minikmeans =MiniBatchKMeans(n_clusters=2,max_no_improvement=20,reassignment_ratio=0.5).fit(X)
    pred = minikmeans.labels_
    if pred[0]==0:
        break
    
flaten_pred = predict_morse(flatten_pred(pred),step)
straight_pred = predict_morse(pred,step)
print('flaten_pred ',flaten_pred)
print('straight_pred ',straight_pred)

# %%
pred_to_label(pred,step,'pred.txt')

# %%
pred_to_label(flatten_pred(pred), step,'flatten_pred.txt')

# %%

# %%
morse_to_text('.- .-')

flaten_pred  - ..-. . .... -. ... . --. .... - -. ... . ... --- - .... - - .... - -.  . . ... . ... . .. .... . . . --...
straight_pred  - ..-. . .... -. ... . --. .... - -. ... . ... --- - .... - - .... - -.  . . ... . ... . .. .... . . . --...


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: Futur

flaten_pred  - ..-. . .... . . ... . --. .... - -. ... . ... --- . .... - - .... - -.  . . ... . . .. . .. .... . . . --...
straight_pred  - ..-. . .... . . ... . --. .... - -. ... . ... --- . .... - - .... - -.  . . ... . . .. . .. .... . . . --...


'A A '

In [13]:
!pip3 install --upgrade pyinstaller


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!pyinstaller --onefile '/home/m.py'

The 'pathlib' package is an obsolete backport of a standard library package and is incompatible with PyInstaller. Please remove this package (located in /usr/local/lib/python3.9/dist-packages) using
    "/usr/bin/python3" -m pip uninstall pathlib
then try again.


In [15]:
!pip install cx_freeze

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 31.8 MB/s eta 0:00:00


In [16]:
!python '/home/setup.py'

Traceback (most recent call last):
  File "/home/setup.py", line 18, in <module>
    executables=[Executable("m.py", base=base)],
  File "/usr/local/lib/python3.9/dist-packages/cx_Freeze/executable.py", line 53, in __init__
    self.base = base
  File "/usr/local/lib/python3.9/dist-packages/cx_Freeze/executable.py", line 95, in base
    raise ConfigError(f"no base named {name!r} ({name_base!r})")
cx_Freeze.exception.ConfigError: no base named 'Win32GUI' ('Win32GUI-cpython-39-x86_64-linux-gnu')


In [17]:
!sudo apt-get install mingw-w64

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binutils-mingw-w64-i686 binutils-mingw-w64-x86-64 g++-mingw-w64
  g++-mingw-w64-i686 g++-mingw-w64-x86-64 gcc-mingw-w64 gcc-mingw-w64-base
  gcc-mingw-w64-i686 gcc-mingw-w64-x86-64 mingw-w64-common mingw-w64-i686-dev
  mingw-w64-x86-64-dev
Suggested packages:
  gcc-9-locales wine wine64
The following NEW packages will be installed:
  binutils-mingw-w64-i686 binutils-mingw-w64-x86-64 g++-mingw-w64
  g++-mingw-w64-i686 g++-mingw-w64-x86-64 gcc-mingw-w64 gcc-mingw-w64-base
  gcc-mingw-w64-i686 gcc-mingw-w64-x86-64 mingw-w64 mingw-w64-common
  mingw-w64-i686-dev mingw-w64-x86-64-dev
0 upgraded, 13 newly installed, 0 to remove and 24 not upgraded.
Need to get 148 MB of archives.
After this operation, 875 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 binutils-mingw-w64-i686 amd64 2.34-5ub

In [19]:
!CC=i686-w64-mingw32-gcc CXX=i686-w64-mingw32-g++ python '/home/setup.py' build

Traceback (most recent call last):
  File "/home/setup.py", line 18, in <module>
    executables=[Executable("m.py", base=base)],
  File "/usr/local/lib/python3.9/dist-packages/cx_Freeze/executable.py", line 53, in __init__
    self.base = base
  File "/usr/local/lib/python3.9/dist-packages/cx_Freeze/executable.py", line 95, in base
    raise ConfigError(f"no base named {name!r} ({name_base!r})")
cx_Freeze.exception.ConfigError: no base named 'Win32GUI' ('Win32GUI-cpython-39-x86_64-linux-gnu')


In [21]:
!CC=i686-w64-mingw32-gcc CXX=i686-w64-mingw32-g++ python '/home/setup.py' build_exe --target-name MyApp.exe --base-name Win32GUI


Traceback (most recent call last):
  File "/home/setup.py", line 18, in <module>
    executables=[Executable("m.py", base=base)],
  File "/usr/local/lib/python3.9/dist-packages/cx_Freeze/executable.py", line 53, in __init__
    self.base = base
  File "/usr/local/lib/python3.9/dist-packages/cx_Freeze/executable.py", line 95, in base
    raise ConfigError(f"no base named {name!r} ({name_base!r})")
cx_Freeze.exception.ConfigError: no base named 'Win32GUI' ('Win32GUI-cpython-39-x86_64-linux-gnu')


In [22]:
!pip install py2exe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 4.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [23]:
!pip install nuitka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.1 MB/s eta 0:00:00
  Created wheel for nuitka: filename=Nuitka-1.5.5-cp39-cp39-linux_x86_64.whl size=2878776 sha256=000dcd090f6881d6009c70632c0d8f2680a70551b916435bb24a827545a3e0b2
  Stored in directory: /root/.cache/pip/wheels/80/c3/45/e0adbafdb89264f5b711bb25c1777e36e4d7449cdf94e04ee1
Successfully built nuitka


In [24]:
!sudo apt-get install gcc-mingw-w64-x86-64 g++-mingw-w64-x86-64


Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++-mingw-w64-x86-64 is already the newest version (9.3.0-7ubuntu1+22~exp1ubuntu4).
g++-mingw-w64-x86-64 set to manually installed.
gcc-mingw-w64-x86-64 is already the newest version (9.3.0-7ubuntu1+22~exp1ubuntu4).
gcc-mingw-w64-x86-64 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [25]:
!nuitka --mingw64 'home/m.py'

/bin/bash: nuitka: command not found


In [26]:
!pip show nuitka


Name: Nuitka
Version: 1.5.5
Summary: Python compiler with full language support and CPython compatibility
Home-page: https://nuitka.net
Author: Kay Hayen
Author-email: Kay.Hayen@gmail.com
License: Apache License, Version 2.0
Location: /usr/local/lib/python3.9/dist-packages
Requires: ordered-set, zstandard
Required-by: 


In [ ]:
!python -m nuitka --standalone --debug '/home/m.py' --mingw64

Nuitka-Options:INFO: Used command line options: --standalone --debug /home/m.py --mingw64
Nuitka-Options:WARNING: Requesting Windows specific compilers has no effect on other platforms.
Nuitka:INFO: Starting Python compilation with Nuitka '1.5.5' on Python '3.9' commercial grade 'not installed'.
Nuitka-Plugins:WARNING: anti-bloat: Undesirable import of 'numba' at '/usr/local/lib/python3.9/dist-packages/pandas/core/_numba/kernels/var_.py:11' encountered. It may slow down compilation.
Nuitka-Plugins:WARNING:     Complex topic! More information can be found at https://nuitka.net/info/unwanted-module.html
Nuitka-Plugins:WARNING: options-nanny: Using module (numba (0, 56, 4)) with no support: Numba is not yet working with Nuitka standalone, recommended to use '--noinclude-numba-mode'
Nuitka-Plugins:INFO: multiprocessing: Injecting pre-module load code for module 'multiprocessing':
Nuitka-Plugins:INFO: multiprocessing:     Monkey patching "multiprocessing" load environment.
Nuitka-Plugins:IN

In [30]:
!nano ~/.bashrc


/bin/bash: nano: command not found
